In [1]:
#https://data.mendeley.com/datasets/5y9wdsg2zt/1 (data)
import os, shutil

In [2]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_pos_dir = '/home/rongzhen/chalmers/digital_twin/crack_image/Positive'
original_dataset_neg_dir = '/home/rongzhen/chalmers/digital_twin/crack_image/Negative'

# The directory where we will
# store our smaller dataset
base_dir = '/home/rongzhen/chalmers/digital_twin/crack_image/CNN_SMALL_GPU'
if os.path.exists(base_dir):
  os.system("rm -rf {}".format(base_dir))
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training posive pictures
train_pos_dir = os.path.join(train_dir, 'pos')
os.mkdir(train_pos_dir)

# Directory with our training negative pictures
train_neg_dir = os.path.join(train_dir, 'neg')
os.mkdir(train_neg_dir)

# Directory with our validation positive pictures
validation_pos_dir = os.path.join(validation_dir, 'pos')
os.mkdir(validation_pos_dir)

# Directory with our validation negative pictures
validation_neg_dir = os.path.join(validation_dir, 'neg')
os.mkdir(validation_neg_dir)

# Directory with our validation positive pictures
test_pos_dir = os.path.join(test_dir, 'pos')
os.mkdir(test_pos_dir)

# Directory with our validation negative pictures
test_neg_dir = os.path.join(test_dir, 'neg')
os.mkdir(test_neg_dir)

In [3]:
# change file name such as 10000_1.jpg to 10000.jpg
for f in os.listdir(original_dataset_pos_dir):
    fname_split = f.split("_")
    if len(fname_split)!=1:
        src = os.path.join(original_dataset_pos_dir,f)
        dst = os.path.join(original_dataset_pos_dir,fname_split[0]+'.jpg')
        os.rename(src, dst) 

for f in os.listdir(original_dataset_neg_dir):
    fname_split = f.split("_")
    if len(fname_split)!=1:
        src = os.path.join(original_dataset_neg_dir,f)
        dst = os.path.join(original_dataset_neg_dir,fname_split[0]+'.jpg')
        os.rename(src, dst) 

# Note: may need to shuffle the images then copy to train, test, validation

In [4]:
# Copy first 14000 images to train
fnames = ['{0:05}.jpg'.format(i) for i in range(1,14001) ]
for fname in fnames:
    src_pos = os.path.join(original_dataset_pos_dir, fname)
    src_neg = os.path.join(original_dataset_neg_dir, fname)

    dst_pos = os.path.join(train_pos_dir, fname)
    dst_neg = os.path.join(train_neg_dir, fname)

    shutil.copyfile(src_pos, dst_pos)
    shutil.copyfile(src_neg, dst_neg)

# Copy first 4000 images to validation
fnames = ['{0:05}.jpg'.format(i) for i in range(14001,18001) ]
for fname in fnames:
    src_pos = os.path.join(original_dataset_pos_dir, fname)
    src_neg = os.path.join(original_dataset_neg_dir, fname)

    dst_pos = os.path.join(validation_pos_dir, fname)
    dst_neg = os.path.join(validation_neg_dir, fname)

    shutil.copyfile(src_pos, dst_pos)
    shutil.copyfile(src_neg, dst_neg)

    
# Copy first 2000 images to test
fnames = ['{0:05}.jpg'.format(i) for i in range(18001,20001) ]
for fname in fnames:
    src_pos = os.path.join(original_dataset_pos_dir, fname)
    src_neg = os.path.join(original_dataset_neg_dir, fname)

    dst_pos = os.path.join(test_pos_dir, fname)
    dst_neg = os.path.join(test_neg_dir, fname)

    shutil.copyfile(src_pos, dst_pos)
    shutil.copyfile(src_neg, dst_neg)
    
  

In [5]:
print('total training pos images:', len(os.listdir(train_pos_dir)))
print('total training neg images:', len(os.listdir(train_neg_dir)))

total training pos images: 14000
total training neg images: 14000


In [6]:
print('total test pos images:', len(os.listdir(test_pos_dir)))
print('total test neg images:', len(os.listdir(test_neg_dir)))

total test pos images: 2000
total test neg images: 2000


In [7]:
print('total validation pos images:', len(os.listdir(validation_pos_dir)))
print('total validation neg images:', len(os.listdir(validation_neg_dir)))

total validation pos images: 4000
total validation neg images: 4000


In [8]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu',
                        input_shape=(227, 227, 3)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 225, 225, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 73, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________

In [10]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [16]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(227, 227),
        batch_size=28,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(227, 227),
        batch_size=2,
        class_mode='binary')

Found 28000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.


In [12]:
train_dir

'/home/rongzhen/chalmers/digital_twin/crack_image/CNN_SMALL_GPU/train'

In [13]:
validation_dir

'/home/rongzhen/chalmers/digital_twin/crack_image/CNN_SMALL_GPU/validation'

In [14]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (28, 227, 227, 3)
labels batch shape: (28,)


In [17]:
#import tensorflow as tf
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

history = model.fit_generator(
      train_generator,
      steps_per_epoch=1000,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=8000)

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[28,16,225,225] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_1/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/acc/Mean_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
model.save('crack_manual_small_gpu.h5')

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from keras import backend as K
K.clear_session()